In [1]:
import os
import sys
current_nb_path = os.getcwd()
project_root = os.path.abspath(os.path.join(current_nb_path, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
os.chdir(project_root)
print(f"🚀 工作目录已切换至: {os.getcwd()}")
print(f"📂 根目录已加入 Path: {project_root}")
import datetime
import torch
import torch.nn.functional as F
from torch.func import jacrev, functional_call
import numpy as np
import matplotlib.pyplot as plt
import wandb
import json
from accelerate import Accelerator
from tqdm.auto import tqdm

from src.model import GeoDCD 
from src.dataloader import get_data_context, load_from_disk, CausalTimeSeriesDataset
from src.visualize import create_dynamic_gif
from src.metrics import count_accuracy
from types import SimpleNamespace

from src.dataloader import get_data_context
args = SimpleNamespace(
    model_path="./results/GD513/20251205_175826/model.pth",
    dataset="GD513",
    data_path="data/real",
    replica_id=0,          # 默认值
    N=513,                 # 你的自定义值
    norm_coords=False,     # 对应 action="store_true"，默认为 False
    window_size=13,        # 你的自定义值
    hierarchy=[32, 8],     # 注意这里直接写列表，而不是字符串
    d_model=64,            # 默认值
    num_bases=4,           # 默认值
    output_dir="./results",# 默认值
    seed=42                # 默认值
)

# 后续的路径构建逻辑保持不变
# Path construction: results/Dataset/Time/inference
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
# 注意：这里需要重新赋值给 args 对象
args.output_dir = os.path.join(args.output_dir, args.dataset, timestamp, "inference")

print(f"配置已就绪，Hierarchy: {args.hierarchy}")

🚀 工作目录已切换至: /mnt/data1/yhy_data/GeoDCD
📂 根目录已加入 Path: /mnt/data1/yhy_data/GeoDCD
配置已就绪，Hierarchy: [32, 8]


In [2]:
_, _, meta = get_data_context(args)
    
model = GeoDCD(
    N=args.N, 
    coords=meta['coords'], 
    hierarchy=args.hierarchy, 
    d_model=args.d_model,
    num_bases=args.num_bases
)
model.eval()
with torch.no_grad():
    dummy_x = torch.zeros(1, args.N, args.window_size)
    model(dummy_x)

📂 Loading GD513 (Replica 0)...
⚠️ Warning: Ground Truth not found at data/real/GD513/gt_0.npy. Metrics will be skipped.
🛡️ Using raw coordinates (Normalization Skipped).
✅ Data Split: Train=1048 samples, Val=261 samples
🔲 Geometric Window Initialized: N=513 -> K=32
🔲 Geometric Window Initialized: N=32 -> K=8


/home/yhy/my_project/GeoDCD/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [3]:
est_fine = model.layers[0].graph.get_soft_graph().detach().cpu().numpy()
est_dynamic = np.load("results/GD513/20251205_175826/est_dynamic.npy")
print(est_dynamic.shape)
patch_ids = np.zeros(est_fine.shape[0])

if hasattr(model, 'structure_S_0'):
    S_matrix = model.structure_S_0.detach().cpu().numpy() 
    if S_matrix.sum() > 0:
        patch_ids = S_matrix.argmax(axis=1)
coords = meta['coords']




# fig = plt.figure(figsize=(20, 15))

# ax1 = fig.add_subplot(1, 1, 1)
# ax1.scatter(coords[:, 0], coords[:, 1], c=patch_ids, cmap='tab20', s=20)
# ax1.set_title(f"Swin Windows (K={len(np.unique(patch_ids))})")

(513, 513, 200)


In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely.geometry import Point

# 假设 coords 是 (N, 2)，且为 [经度, 纬度]
# 如果是归一化坐标，你需要先反归一化回经纬度
geometry = [Point(xy) for xy in coords]
gdf = gpd.GeoDataFrame({'patch_id': patch_ids}, geometry=geometry, crs="EPSG:4326")

# 转换坐标系以适配底图 (Web Mercator)
gdf_3857 = gdf.to_crs(epsg=3857)
gdf_3857

,patch_id,geometry
0,18,POINT (12930866.086 4855026.597)
1,18,POINT (12932827.473 4853066.967)
2,18,POINT (12933481.269 4853066.967)
3,18,POINT (12934135.064 4853720.134)
4,18,POINT (12934135.064 4855679.893)
...,...,...
508,28,POINT (12981208.353 4855026.597)
509,28,POINT (12981208.353 4855679.893)
510,28,POINT (12981862.148 4855026.597)
511,16,POINT (12983169.739 4864176.655)


In [7]:
from datetime import datetime, timedelta

records = []
T = est_dynamic.shape[2]
N = est_dynamic.shape[0]

start_time = datetime.strptime("2023-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")
time_step_sampling = 1 

for t in range(0, T, time_step_sampling):
    matrix = est_dynamic[:, :, t].copy()
    np.fill_diagonal(matrix, 0)
    K = 50
    flat_indices = np.argpartition(matrix.ravel(), -K)[-K:]
    r_indices, c_indices = np.unravel_index(flat_indices, matrix.shape)
    current_real_time = start_time + timedelta(minutes=int(t) * 10)
    time_str = current_real_time.strftime("%Y-%m-%d %H:%M:%S")
    
    for r, c in zip(r_indices, c_indices):
        val = matrix[r, c]
        if val <= 0: continue 
        
        records.append({
            'source_lat': float(coords[r, 1]), 
            'source_lon': float(coords[r, 0]),
            'target_lat': float(coords[c, 1]), 
            'target_lon': float(coords[c, 0]),
            'value': float(val),
            'timestamp': time_str,  
            'patch_group': int(patch_ids[r])
        })

df_kepler = pd.DataFrame(records)

# 打印预览，确保格式正确
print(f"数据生成完毕: {len(df_kepler)} 条记录")
print(df_kepler[['timestamp', 'value', 'source_lat']].head())

# 导出
df_kepler.to_csv("kepler_data_GD_v2.csv", index=False)

数据生成完毕: 10000 条记录
             timestamp     value  source_lat
0  2023-09-01 00:00:00  0.316406   39.866337
1  2023-09-01 00:00:00  0.316406   39.987849
2  2023-09-01 00:00:00  0.320312   39.852835
3  2023-09-01 00:00:00  0.320312   39.866337
4  2023-09-01 00:00:00  0.324219   39.956346


In [8]:
df_kepler

,source_lat,source_lon,target_lat,target_lon,value,timestamp,patch_group
0,39.866337,116.430111,39.866337,116.447731,0.316406,2023-09-01 00:00:00,12
1,39.987849,116.418365,39.951845,116.441857,0.316406,2023-09-01 00:00:00,4
2,39.852835,116.371380,39.807831,116.342014,0.320312,2023-09-01 00:00:00,11
3,39.866337,116.306775,39.879838,116.271536,0.320312,2023-09-01 00:00:00,24
4,39.956346,116.459477,39.983348,116.488843,0.324219,2023-09-01 00:00:00,0
...,...,...,...,...,...,...,...
9995,39.924842,116.441857,39.906841,116.465350,0.416016,2023-09-02 09:10:00,17
9996,39.893339,116.477096,39.893339,116.482969,0.421875,2023-09-02 09:10:00,27
9997,39.897840,116.424238,39.888839,116.447731,0.425781,2023-09-02 09:10:00,22
9998,39.839334,116.512335,39.884338,116.529955,0.427734,2023-09-02 09:10:00,15
